In [1]:
import os
import time
import logging
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import sys


# Set up logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
log_dir = '/kaggle/working/'
os.makedirs(log_dir, exist_ok=True)
log_file = os.path.join(log_dir, 'efficientnet_model_training.log')
logging.basicConfig(
    filename=log_file,
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)


# Redirect output to a log file
log_file = "/kaggle/working/training_output.log"
sys.stdout = open(log_file, "w")

# Your training code
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=25,
    verbose=1
)

# Restore standard output
sys.stdout.close()
sys.stdout = sys.__stdout__


def create_efficient_net_model(input_shape=(128, 128, 3)):
    # Load pre-trained EfficientNetB0 with imagenet weights
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze the base model layers
    base_model.trainable = False
    
    # Create new model
    inputs = Input(shape=input_shape)
    # Preprocess input
    x = tf.keras.applications.efficientnet.preprocess_input(inputs)
    
    # Base model
    x = base_model(x)
    
    # Add custom layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    return model, base_model

# Data augmentation setup
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

def train_model(model, base_model, train_generator, validation_generator, epochs=50):
    # First phase: Train only the top layers
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
        metrics=['accuracy']
    )
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
        ModelCheckpoint('best_efficientnet_model.keras', monitor='val_loss', save_best_only=True),
        CSVLogger(os.path.join(log_dir, 'efficientnet_training.csv'))
    ]
    
    # First phase training
    history_1 = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=epochs//2,
        callbacks=callbacks
    )
    
    # Second phase: Fine-tune the last few layers of the base model
    # Unfreeze the last 20 layers
    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
        metrics=['accuracy']
    )
    
    # Second phase training
    history_2 = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=epochs//2,
        callbacks=callbacks
    )
    
    return history_1, history_2

# Data generators setup
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/chest-x-ray-images/FinalData/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/chest-x-ray-images/FinalData/val',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/chest-x-ray-images/FinalData/test',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

# Create and train model
model, base_model = create_efficient_net_model()
start_time = time.time()
history_1, history_2 = train_model(model, base_model, train_generator, validation_generator)
execution_time = time.time() - start_time

# Evaluate model
y_true = validation_generator.classes
y_pred = (model.predict(validation_generator) > 0.5).astype("int32")

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Log and print results
logging.info(f'EfficientNet Model - Accuracy: {accuracy:.4f}')
logging.info(f'EfficientNet Model - Precision: {precision:.4f}')
logging.info(f'EfficientNet Model - Recall: {recall:.4f}')
logging.info(f'EfficientNet Model - F1-score: {f1:.4f}')
logging.info(f'EfficientNet Model - Execution Time: {execution_time:.2f} seconds')

print(f"EfficientNet Model - Accuracy: {accuracy:.4f}")
print(f"EfficientNet Model - Precision: {precision:.4f}")
print(f"EfficientNet Model - Recall: {recall:.4f}")
print(f"EfficientNet Model - F1-score: {f1:.4f}")
print(f"EfficientNet Model - Execution Time: {execution_time:.2f} seconds")

Found 13830 images belonging to 2 classes.
Found 166 images belonging to 2 classes.
Found 2096 images belonging to 2 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1732527885.944043     118 service.cc:145] XLA service 0x7e54fc0029c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732527885.944114     118 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1732527885.944118     118 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  1/433 ━━━━━━━━━━━━━━━━━━━━ 4:04:45 34s/step - accuracy: 0.5312 - loss: 0.6887

I0000 00:00:1732527907.075251     118 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


433/433 ━━━━━━━━━━━━━━━━━━━━ 239s 475ms/step - accuracy: 0.7579 - loss: 0.5472 - val_accuracy: 0.8735 - val_loss: 0.4031 - learning_rate: 0.0010
Epoch 2/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 117s 266ms/step - accuracy: 0.8204 - loss: 0.4738 - val_accuracy: 0.8313 - val_loss: 0.4294 - learning_rate: 0.0010
Epoch 3/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 117s 266ms/step - accuracy: 0.8201 - loss: 0.4707 - val_accuracy: 0.9277 - val_loss: 0.3863 - learning_rate: 0.0010
Epoch 4/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 116s 264ms/step - accuracy: 0.8314 - loss: 0.4539 - val_accuracy: 0.9096 - val_loss: 0.3532 - learning_rate: 0.0010
Epoch 5/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 116s 264ms/step - accuracy: 0.8324 - loss: 0.4509 - val_accuracy: 0.9277 - val_loss: 0.3573 - learning_rate: 0.0010
Epoch 6/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 116s 263ms/step - accuracy: 0.8385 - loss: 0.4450 - val_accuracy: 0.9036 - val_loss: 0.3579 - learning_rate: 0.0010
Epoch 7/15
433/433 ━━━━━━━━━━━━━━━━━━━━ 115s 262ms/step - accuracy: 0.8424 

In [1]:
import os
import time
import logging
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import zipfile
import pandas as pd
from datetime import datetime

# Create timestamp for unique run identification
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Set up directories
base_log_dir = '/kaggle/working/model_logs'
run_dir = os.path.join(base_log_dir, f'run_{timestamp}')
model_dir = os.path.join(run_dir, 'models')
plot_dir = os.path.join(run_dir, 'plots')
log_dir = os.path.join(run_dir, 'logs')

# Create directories
for dir_path in [model_dir, plot_dir, log_dir]:
    os.makedirs(dir_path, exist_ok=True)

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(log_dir, 'training.log')),
        logging.StreamHandler(sys.stdout)
    ]
)

def plot_training_history(history1, history2, plot_dir):
    """Plot and save training metrics"""
    # Combine histories
    history_combined = {
        'accuracy': history1.history['accuracy'] + history2.history['accuracy'],
        'val_accuracy': history1.history['val_accuracy'] + history2.history['val_accuracy'],
        'loss': history1.history['loss'] + history2.history['loss'],
        'val_loss': history1.history['val_loss'] + history2.history['val_loss']
    }
    
    # Plot training curves
    plt.figure(figsize=(15, 5))
    
    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history_combined['accuracy'], label='Training Accuracy')
    plt.plot(history_combined['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history_combined['loss'], label='Training Loss')
    plt.plot(history_combined['val_loss'], label='Validation Loss')
    plt.title('Model Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(os.path.join(plot_dir, 'training_history.png'))
    plt.close()

def plot_confusion_matrix(y_true, y_pred, plot_dir):
    """Plot and save confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(os.path.join(plot_dir, 'confusion_matrix.png'))
    plt.close()

def save_metrics_report(metrics, log_dir):
    """Save detailed metrics report"""
    with open(os.path.join(log_dir, 'metrics_report.txt'), 'w') as f:
        for key, value in metrics.items():
            f.write(f"{key}: {value}\n")

def create_efficient_net_model(input_shape=(128, 128, 3)):
    # Load pre-trained EfficientNetB0 with imagenet weights
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    # Freeze the base model layers
    base_model.trainable = False
    
    # Create new model
    inputs = Input(shape=input_shape)
    # Preprocess input
    x = tf.keras.applications.efficientnet.preprocess_input(inputs)
    
    # Base model
    x = base_model(x)
    
    # Add custom layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    return model, base_model

# Data augmentation setup
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

def train_model(model, base_model, train_generator, validation_generator, epochs=50):
    # First phase: Train only the top layers
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
        metrics=['accuracy']
    )
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
        ModelCheckpoint('best_efficientnet_model.keras', monitor='val_loss', save_best_only=True),
        CSVLogger(os.path.join(log_dir, 'efficientnet_training.csv'))
    ]
    
    # First phase training
    history_1 = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=epochs//2,
        callbacks=callbacks
    )
    
    # Second phase: Fine-tune the last few layers of the base model
    # Unfreeze the last 20 layers
    base_model.trainable = True
    for layer in base_model.layers[:-20]:
        layer.trainable = False
    
    # Recompile with lower learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1),
        metrics=['accuracy']
    )
    
    # Second phase training
    history_2 = model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=epochs//2,
        callbacks=callbacks
    )
    
    return history_1, history_2

# Data generators setup
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/chest-x-ray-images/FinalData/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/kaggle/input/chest-x-ray-images/FinalData/val',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

test_generator = test_datagen.flow_from_directory(
    '/kaggle/input/chest-x-ray-images/FinalData/test',
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)
def create_results_archive(run_dir, timestamp):
    """Create zip archive of all results"""
    zip_path = f'/kaggle/working/model_results_{timestamp}.zip'
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(run_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, run_dir)
                zipf.write(file_path, arcname)
    return zip_path

# Main execution
if __name__ == "__main__":
    logging.info("Starting model training pipeline")
    
    try:
        # Create and train model
        model, base_model = create_efficient_net_model()
        logging.info("Model created successfully")
        
        start_time = time.time()
        history_1, history_2 = train_model(model, base_model, train_generator, validation_generator)
        execution_time = time.time() - start_time
        
        # Evaluate model
        y_true = validation_generator.classes
        y_pred = (model.predict(validation_generator) > 0.5).astype("int32")
        
        # Compute metrics
        metrics = {
            'Accuracy': accuracy_score(y_true, y_pred),
            'Precision': precision_score(y_true, y_pred),
            'Recall': recall_score(y_true, y_pred),
            'F1-score': f1_score(y_true, y_pred),
            'Execution Time': f"{execution_time:.2f} seconds"
        }
        
        # Generate plots and save results
        plot_training_history(history_1, history_2, plot_dir)
        plot_confusion_matrix(y_true, y_pred, plot_dir)
        save_metrics_report(metrics, log_dir)
        
        # Create zip archive
        zip_path = create_results_archive(run_dir, timestamp)
        
        logging.info("Training pipeline completed successfully")
        logging.info(f"Results archived at: {zip_path}")
        
        # Print final metrics
        for metric, value in metrics.items():
            print(f"{metric}: {value}")
            
    except Exception as e:
        logging.error(f"Error during execution: {str(e)}")
        raise

Found 13830 images belonging to 2 classes.
Found 166 images belonging to 2 classes.
Found 2096 images belonging to 2 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/25


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1732533340.486769     117 service.cc:145] XLA service 0x7f31900016a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732533340.486821     117 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  1/433 ━━━━━━━━━━━━━━━━━━━━ 3:51:06 32s/step - accuracy: 0.5312 - loss: 0.7080

I0000 00:00:1732533359.876210     117 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


433/433 ━━━━━━━━━━━━━━━━━━━━ 243s 488ms/step - accuracy: 0.7653 - loss: 0.5444 - val_accuracy: 0.8976 - val_loss: 0.3934 - learning_rate: 0.0010
Epoch 2/25
433/433 ━━━━━━━━━━━━━━━━━━━━ 115s 262ms/step - accuracy: 0.8190 - loss: 0.4754 - val_accuracy: 0.9398 - val_loss: 0.3645 - learning_rate: 0.0010
Epoch 3/25
433/433 ━━━━━━━━━━━━━━━━━━━━ 112s 254ms/step - accuracy: 0.8212 - loss: 0.4669 - val_accuracy: 0.8855 - val_loss: 0.4061 - learning_rate: 0.0010
Epoch 4/25
433/433 ━━━━━━━━━━━━━━━━━━━━ 112s 253ms/step - accuracy: 0.8269 - loss: 0.4566 - val_accuracy: 0.8916 - val_loss: 0.3984 - learning_rate: 0.0010
Epoch 5/25
433/433 ━━━━━━━━━━━━━━━━━━━━ 115s 261ms/step - accuracy: 0.8242 - loss: 0.4549 - val_accuracy: 0.8976 - val_loss: 0.3632 - learning_rate: 0.0010
Epoch 6/25
433/433 ━━━━━━━━━━━━━━━━━━━━ 113s 257ms/step - accuracy: 0.8304 - loss: 0.4527 - val_accuracy: 0.9096 - val_loss: 0.3576 - learning_rate: 0.0010
Epoch 7/25
433/433 ━━━━━━━━━━━━━━━━━━━━ 113s 257ms/step - accuracy: 0.8348 

In [1]:
import shutil

# Compress the output directory
shutil.make_archive("/kaggle/working/model_logs", 'zip', "/kaggle/working/")


'/kaggle/working/model_logs.zip'

In [4]:
from IPython.display import FileLink 
FileLink(r'model_logs.zip')

/kaggle/working/model_logs.zip